In [1]:
from pathlib import Path
from pydantic.v1 import BaseModel, Field

from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [16]:
save_dir = Path("data", "standardisation")
summaries_dir = Path("data", "summaries")
sections_2015_fname = Path(summaries_dir, "2015_sections_summary.txt")
sections_2023_fname = Path(summaries_dir, "2023_sections_summary.txt")

In [6]:
with open(sections_2015_fname, "r") as f:
    section_2015 = f.read()
with open(sections_2023_fname, "r") as f:
    section_2023 = f.read()

In [11]:
prompt = """You are an expert lawyer with a speciality in contract law. 
Please provide advice on what a common standardised format would look like for the 
following texts. The format should include sections with headings with a brief description of the section 
and should capture all the information present in both texts. 
The first text includes information from 2015 and is contained with three backticks (```). 
The second text contains information from 2023 and is contained wihin three hash symbols (###).

                      
2015 text: ```{text1}```
            
2023 text: ###{text2}###
"""

prompt_template = PromptTemplate(input_variables=["text1", "text2"], template=prompt)

In [19]:
gpt35 = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
gpt4 = ChatOpenAI(temperature=0, model="gpt-4")

In [13]:
human_message_prompt = HumanMessagePromptTemplate.from_template(prompt)
chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

In [14]:
gpt_prompt_engineering_output = gpt35(
    chat_prompt.format_prompt(text1=section_2015, text2=section_2023).to_messages()
)

In [15]:
print(gpt_prompt_engineering_output.content)

Standardized Format:

1. Introduction
   - Brief description: Provides an overview of the agreement and the services covered.
   - Information from 2015: Outlines the terms and conditions for the iTunes Store Service provided by iTunes SARL.
   - Information from 2023: Outlines terms and conditions for using Apple's Services such as the App Store, Apple Music, Apple TV, among others.

2. Payment and Purchases
   - Brief description: Covers payment methods, cancellation policies, and refunds.
   - Information from 2015: Users must pay for all products purchased through the iTunes store and provide a valid payment method. Purchases can be cancelled within 14 days of receipt, except for iTunes gifts which cannot be refunded once the code is redeemed. If a product becomes unavailable after purchase but before download, the user is entitled to a refund of the price paid. Pre-orders can be cancelled before the product becomes available. iTunes reserves the right to change product prices at a

In [18]:
with open(Path(save_dir, "gpt35_standardisation_advice.txt"), "w") as f:
    f.write(gpt_prompt_engineering_output.content)

# GPT 4

In [20]:
gpt_prompt_engineering_output = gpt4(
    chat_prompt.format_prompt(text1=section_2015, text2=section_2023).to_messages()
)

In [22]:
print(gpt_prompt_engineering_output.content)

Based on the information provided in both texts, a common standardised format for the contract could be as follows:

1. **Introduction**
   - Brief overview of the agreement and the parties involved.

2. **Eligibility and Account Creation**
   - Age requirements for account creation.
   - Responsibility for account security and activities.
   - Limitations on the number of devices and Apple IDs per device.

3. **Services and Content**
   - Description of services and content offered.
   - Requirements for using the services (compatible hardware, software, internet access).
   - Limitations on service availability (geographical restrictions, device restrictions).

4. **Purchasing and Payment**
   - Payment methods and charges, including taxes.
   - Rules for pre-orders and cancellations.
   - Details about automatic renewals and subscription-based content.
   - Information about special offerings like Season Pass, Multi-Pass, Series Pass.

5. **User Responsibilities**
   - Rules for per

In [21]:
with open(Path(save_dir, "gpt4_standardisation_advice.txt"), "w") as f:
    f.write(gpt_prompt_engineering_output.content)